In [1]:
import org.apache.spark._ 
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Row, SparkSession}

Intitializing Scala interpreter ...

Spark Web UI available at http://0a53d4451f83:4041
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591425630672)
SparkSession available as 'spark'


2020-06-06 06:40:29,844 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-06-06 06:40:30,596 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


import org.apache.spark._
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Row, SparkSession}


In [2]:
val df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS-clean-FINAL.csv")

df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [3]:
val df_num = df.drop("Date","Location","WindGustDir","WindDir9am","WindDir3pm","RainToday","RainTomorrow",
                     "RISK_MM","Month","Pressure9am","Pressure3pm","date_Sc","date-day_Sc","date-month_Sc","date-year_Sc",
                     "Evaporation","Sunshine","Cloud9am","Cloud3pm","Pressure3pm_Sc","Pressure9am_Sc","WindGustSpeed")
                .withColumn("RainToday_Sc",col("RainToday_Sc").cast(DoubleType))
                .withColumn("RainTomorrow_Sc",col("RainTomorrow_Sc").cast(DoubleType))
df_num.printSchema()

root
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday_Sc: double (nullable = true)
 |-- RainTomorrow_Sc: double (nullable = true)



df_num: org.apache.spark.sql.DataFrame = [MinTemp: double, MaxTemp: double ... 9 more fields]


In [4]:
// convert to vector column first
val vector_col = "corr_features"
val assembler = new VectorAssembler().setInputCols(df_num.columns.toArray).setOutputCol(vector_col)
val df_vector = assembler.transform(df_num).select(vector_col)

// get correlation matrix
val Row(coeff1: Matrix) = Correlation.corr(df_vector,vector_col).head

2020-06-06 06:40:48,633 WARN  [Executor task launch worker for task 6] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-06 06:40:48,634 WARN  [Executor task launch worker for task 6] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


vector_col: String = corr_features
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_86e749d0d66f
df_vector: org.apache.spark.sql.DataFrame = [corr_features: vector]
coeff1: org.apache.spark.ml.linalg.Matrix =
1.0                    0.736773319224934     ... (11 total)
0.736773319224934      1.0                   ...
0.1032747100008999     -0.07375231160514448  ...
0.17361779475707115    0.01309910338554134   ...
0.17271467643471622    0.04975583636448577   ...
-0.23576460688378328   -0.5044486150424999   ...
1.6918117304553726E-4  -0.5086602899657561   ...
0.9019093470536083     0.8876454668089747    ...
0.7123737658823907     0.9828715270649254    ...
0.05525968787984383    -0.22668484736527175  ...
0.08336903898009859    -0.15945177018161483  ...


In [5]:
val cols = (0 until coeff1.numCols).toSeq
val col_names = df_num.columns.toArray

val df = coeff1.transpose
  .colIter.toSeq
  .map(_.toArray)
  .toDF("arr")

val df2 = cols.foldLeft(df)((df, i) => df.withColumn("_" + (i+1), $"arr"(i)))
  .drop("arr")
  
val df3 = df2.columns.foldLeft(df2){(df,colName) =>df.withColumn(colName,round(col(colName),3))}

val df_CM = df3.toDF(col_names:_*)


cols: scala.collection.immutable.Range = Range(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
col_names: Array[String] = Array(MinTemp, MaxTemp, Rainfall, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Temp9am, Temp3pm, RainToday_Sc, RainTomorrow_Sc)
df: org.apache.spark.sql.DataFrame = [arr: array<double>]
df2: org.apache.spark.sql.DataFrame = [_1: double, _2: double ... 9 more fields]
df3: org.apache.spark.sql.DataFrame = [_1: double, _2: double ... 9 more fields]
df_CM: org.apache.spark.sql.DataFrame = [MinTemp: double, MaxTemp: double ... 9 more fields]


In [6]:
df_CM.show()

+-------+-------+--------+------------+------------+-----------+-----------+-------+-------+------------+---------------+
|MinTemp|MaxTemp|Rainfall|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Temp9am|Temp3pm|RainToday_Sc|RainTomorrow_Sc|
+-------+-------+--------+------------+------------+-----------+-----------+-------+-------+------------+---------------+
|    1.0|  0.737|   0.103|       0.174|       0.173|     -0.236|        0.0|  0.902|  0.712|       0.055|          0.083|
|  0.737|    1.0|  -0.074|       0.013|        0.05|     -0.504|     -0.509|  0.888|  0.983|      -0.227|         -0.159|
|  0.103| -0.074|     1.0|       0.088|       0.057|      0.222|      0.251|   0.01| -0.076|       0.487|          0.245|
|  0.174|  0.013|   0.088|         1.0|       0.516|      -0.27|     -0.031|  0.126| -0.001|       0.101|          0.091|
|  0.173|   0.05|   0.057|       0.516|         1.0|     -0.147|      0.015|  0.162|  0.025|       0.077|          0.086|
| -0.236| -0.504|   0.22